# Import Modules

In [ ]:
import h5py as h5
import pathlib
import importlib

#Methods modules
import numpy as np
from scipy import ndimage
from scipy import linalg as la
import umap

#Display modules
import matplotlib.pyplot as plt
from seaborn import heatmap
import matplotlib.cm as cm

# User defined modules
import userfunctions as uf
import clusteranalysis as cn


# Import Files

In [ ]:
# Define main path
main_path = pathlib.Path().absolute()

#or input path manually
#main_path = input('Main working directory: ')

In [ ]:
#data_name = 'bevstatep8_9'
data_name = input('Name of dataset to be used: ')
datadir = main_path+data_name
data = h5.File(datadir,'r')
print(list(data.keys()))

The dataset used is an .h5 file that should contain the kinematic variables displayed by the key. Respectively

    - 'Average distance'
    - 'Alignment' - associated with to heading alignment
    - 'acc Alignment' - associated to the acceleration alignment
    - 'Mean speed'
    - 'mean tail beating rate'
    - 'var tail beating rate' -  from which the deviation is computed 

In [ ]:
X1 = np.zeros((6,len(data['Alignment'])))

#Normalize the variables used to the interval between 0 and 1

X1[0] = data['Average distance']/np.max(data['Average distance'])
X1[3] = data['Mean speed']/np.max(data['Mean speed'])
X1[4,:-1] = data['mean tail beating rate']/np.max(data['mean tail beating rate'])

#Transform alignment variables
X1[1] = np.arccos(data['Alignment'])/np.pi
X1[2] =np.arccos(data['acc alignment'])/np.pi

#compute the deviation of the tail beating rate
X1[5,:-1] = np.sqrt(data['var tail beating rate'][:])/data['mean tail beating rate']

X = np.nan_to_num(X1)[:,:]

# Embedding process and cluster analysis

## Select embedding sample 

In [ ]:
N = 100000
rand_vec1 = uf.min_dist_vec(X,N)
rand_vec2 = rand_vec1[np.array(np.sort(np.random.rand(4000)*(len(rand_vec1)-1)),'int32')]

## Embedding

### Run if saved embedding available

In [ ]:
embed_load_name = input('What is the embedding file to be opened?: ')
embed_data = embed_load_name + '.h5'

In [ ]:
#embed_data = 'embp8_9.h5'
embeddingA1_file = h5.File(main_path+embed_data,'r')
embeddingA1 = embeddingA1_file['data']

### Run if saved embedding not available

In [ ]:
lr = input('What is the learning_rate to be used: ')

#reducer1 = umap.UMAP(learning_rate=7.0,random_state=42)
reducer1 = umap.UMAP(learning_rate=lr,random_state=42)
reducer1.fit(X[:,rand_vec2].T)
embeddingA1 = reducer1.transform(X[:,rand_vec1].T)

#### Save embedding for later uses 

In [4]:
embed_save_name = input('Name of savefile: ')
embed_save_name = embed_save_name + '.h5'

Name of savefileabc


In [ ]:
#embed_save = h5.File('embp8_9.h5','w')
embed_save = h5.File(embed_save_name,'w')
embed_save.create_dataset('data',data = embeddingA1)
embed_save.create_dataset('learning_rate',data =7)
embed_save.close()

## Find density peaks and do clustering around them

### Density Peaks

In [ ]:
n_bins_kde = int(np.sqrt(len(rand_vec1)))
n_filt_kde = int(np.sqrt(n_bins)/2)
kdebev, bev1, bev2 = np.histogram2d(embeddingA1[:,0],embeddingA1[:,1],bins=int(np.sqrt(len(rand_vec1))))
#f_kdebev = ndimage.gaussian_filter(kdebev,10)
f_kdebev = ndimage.gaussian_filter(kdebev,n_filt_kde)

In [13]:
#Dby, Dbx = uf.diff_2d(f_kdebev)
#Dbxy, Dbxx = uf.diff_2d(Dbx)
#Dbyy, Dbyx = uf.diff_2d(Dby)
MX = uf.peaks_2D(f_kdebev)

7.0710678118654755

### Cluster Labelling

In [ ]:
thpca01 = (MX) > 0.05*(np.max(MX))
dil_thpca = (morphology.binary_dilation(thpca01))
blobs = measure.label(dil_thpca)
qthr1, qthr2, res, thr = cn.kmeans_cluster_bev(embeddingA1,blobs) 

### Temporal Assignment

In [ ]:
N = len(res)
bevmat = np.zeros((int(len(X[0])),N))

for i in range(bevmat.shape[1]):
    bevmat[np.arange(int(len(X[0])))[rand_vec1][res[i]],i] = 1
    
r_1 = int(len(X[0])/len(rand_vec1))
bevmat_conv = np.zeros(bevmat.shape)
for i in range(len(rand_vec1)):
    bevmat_conv[i*r_1:(i+1)*r_1] = bevmat[rand_vec1[i]]

## Symbolic Analysis 

### Cluster Labelling Process

In [ ]:
behav_label = cn.cluster_label_behaviors(X,bevmat_conv) 

pas_disp = behav_label[0]
act_disp = behav_label[1]
sym_fight = behav_label[2]
asym_fight = behav_label[3]
freeze = behav_label[4]
other = behav_label[5]

In [ ]:
# Shift order of bevmat_conv
bevmat_conv2 = cn.reorder_label_bevmat(behav_label,bevmat_conv)

In [ ]:
symbseq = cn.symbol_seq(bevmat_conv2)
compseq,t_dwell = cn.compress_seq(symbseq)
avg_td, std_td = cn.t_dwell_dist(t_dwell,bevmat.shape[1],0.01)

In [ ]:
avg_var = cn.cluster_bev_avg(X,bevmat_conv2)

In [ ]:
as_fight_c1 = np.concatenate(np.array(qthr1)[asym_fight[:]],axis=0)
as_fight_c2 = np.concatenate(np.array(qthr2)[asym_fight[:]],axis=0)
sym_fight_c1 = np.concatenate(np.array(qthr1)[sym_fight[:]],axis=0)
sym_fight_c2 = np.concatenate(np.array(qthr2)[sym_fight[:]],axis=0)
freeze_c1 = np.concatenate(np.array(qthr1)[freeze],axis=0)
freeze_c2 = np.concatenate(np.array(qthr2)[freeze],axis=0)
pdisplay_c1 = np.concatenate(np.array(qthr1)[pas_disp[:]],axis=0)
pdisplay_c2 = np.concatenate(np.array(qthr2)[pas_disp[:]],axis=0)
adisplay_c1 = np.concatenate(np.array(qthr1)[act_disp],axis=0)
adisplay_c2 = np.concatenate(np.array(qthr2)[act_disp],axis=0)
other_c1 = np.concatenate(np.array(qthr1)[other],axis=0)
other_c2 = np.concatenate(np.array(qthr2)[other],axis=0)


### Eigenspectrum and dynamics

In [ ]:
# Compute transition matrix
Msymb = cn.transition_matrix_tau(compseq,1)
m1_2 = cn.prop_trans_mat(Msymb)

In [ ]:
# Compute eigenvalue for different transitions
eig_tau, ent2_v,H_v = cn.process_analysis(compseq,bevmat_conv2.shape[1])

In [ ]:
# Compute eigenvalues for a random process
rnd_seq = uf.gen_rand_seq(compseq)
eig_tau_rnd, ent2_v_rnd,H_v_rnd = cn.process_analysis(rnd_seq,bevmatconv2.shape[1])

In [ ]:
# Compute eigenvectors
eig_sys = la.eig(np.nan_to_num(m1_2),left=False)
eig_v = np.abs(eig_sys[0])
eig_u1 = eig_sys[1].T
loc_eig = np.argsort(eig_v)[::-1]
eig_u1 = eig_u1[loc_eig]
eig_u = np.dot(eig_u1,m1_2)

# Visualization

## Kernel Density Estimate

In [ ]:
plt.figure(figsize=(14,14))

plt.title('KDE estimate',fontsize=20)
plt.pcolormesh(bev1,bev2,(f_kdebev),cmap='jet')
plt.colorbar()
plt.xlabel('Z1',fontsize=15)
plt.ylabel('Z2',fontsize=15)

## Averages in clusters

In [ ]:
max_tb = np.max(data['mean tail beating rate'])
max_d = np.max(data['Average distance'])
max_s = np.max(data['Mean speed'])
max_vtb = np.max(data['var tail beating rate'])

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(231)
plt.bar(np.arange(1,N+1),avg_var[4]*max_tb,alpha=0.5,yerr=avg_var[4]*max_tb/10,color='gold')
plt.title('Mean 2$\pi\omega(s^{-1})$',fontsize=20)
plt.grid()

plt.subplot(232)
plt.bar(np.arange(1,N+1),avg_var[0]*max_d,alpha=0.5,yerr=avg_var[0]*max_d/10,color='orangered')
plt.title('distance (bl)',fontsize=20)
plt.grid()

plt.subplot(233)
plt.bar(np.arange(1,N+1),avg_var[3]*max_s,alpha=0.5,yerr=avg_var[3]*max_s/10,color='cornflowerblue')
plt.title('speed (bl/s)',fontsize=20)
plt.grid()

plt.subplot(234)
plt.bar(np.arange(1,N+1),avg_var[1],alpha=0.5,yerr=avg_var[1]/10,color='green')
plt.title('alignment',fontsize=20)

plt.subplot(235)
plt.bar(np.arange(1,N+1),avg_var[2],alpha=0.5,yerr=avg_var[2]/10,color='limegreen')
plt.title('Acc. alignment',fontsize=20)

plt.subplot(236)
plt.bar(np.arange(1,N+1), avg_var[5], alpha=0.5,yerr=avg_var[5]/10,color='goldenrod')
plt.title('Dev $\omega$',fontsize=20)
plt.ylim(0.0,0.02)
plt.plot
plt.grid()

## Transition Matrix 

In [ ]:
plt.figure(figsize=(12,10))
plt.subplot(111)
heatmap(np.nan_to_num(m1_2),annot=False,linewidth=1,cmap='Reds',vmin = 0.0,vmax=0.15)

### Eigenvalues (for different transition steps)

In [ ]:
plt.figure(figsize=(16,8))

tau = np.arange(1,N2+1)

plt.subplot(121)


plt.plot(tau,eig_tau[:,N1-2])
plt.fill_between(tau,eig_tau[:,N1-2]*(1-0.05),eig_tau[:,N1-2]*(1+0.05),alpha=0.3)

plt.plot(tau,eig_tau[:,N1-3])
plt.fill_between(tau,eig_tau[:,N1-3]*(1-0.05),eig_tau[:,N1-3]*(1+0.05),alpha=0.3)

plt.plot(tau,eig_tau[:,N1-4])
plt.fill_between(tau,eig_tau[:,N1-4]*(1-0.05),eig_tau[:,N1-4]*(1+0.05),alpha=0.3)

plt.plot(tau,eig_tau[0,N1-2]**tau,'b--')
plt.plot(tau,eig_tau[0,N1-3]**tau,'r--')
plt.plot(tau,eig_tau[0,N1-4]**tau,'g--')

plt.plot(tau,eig_tau_rnd[:,0],'k--')

plt.legend(['$\lambda_2$','$\lambda_3$','$\lambda_4$','$\lambda_2$ Markov Model','$\lambda_3$ Markov Model','$\lambda_4$ Markov Model','$\lambda$ Random Model'],fontsize=16)
plt.title('Eigenvalue',fontsize=20)
plt.xscale('log')
#plt.yscale('log')
plt.xlabel('$\\tau$ (transitions)',fontsize=18)

plt.subplot(122)



plt.plot(tau,-np.log(eig_tau[:,-2])/tau)
plt.plot(tau,-np.log(eig_tau[:,N1-3])/tau)
plt.plot(tau,-np.log(eig_tau[:,N1-4])/tau)

plt.plot(tau,-np.log(0.01+eig_tau_rnd[:,0])/tau,'k--')
plt.title('Decay rate',fontsize=20)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$\\tau$ (transitions)',fontsize=18)
plt.ylim(1e-4,1)
plt.legend(['$\lambda_2$','$\lambda_3$','$\lambda_4$','$\lambda$ Random Model'],fontsize=16)

### Eigenvectors

In [ ]:
plt.figure(figsize=(16,12))

ld1 = len(pas_disp)
ld2 = len(pas_disp)+len(act_disp)
la2 = ld2 + len(asym_fight)
la1 = la2 + len(sym_fight)
lfr = la1 + len(freeze)
lres = lfr+len(other)

plt.subplot(221)
plt.bar(np.arange(ld1),np.real(eig_u[0])[:ld1],color='b',alpha=0.5)
plt.bar(np.arange(ld1,ld2),np.real(eig_u[0])[ld1:ld2],color='c',alpha=0.5)
plt.bar(np.arange(ld2,la2),np.real(eig_u[0])[ld2:la2],color='m',alpha=0.5)
plt.bar(np.arange(la2,la1),np.real(eig_u[0])[la2:la1],color='r',alpha=0.5)
plt.bar(np.arange(la1,lfr),np.real(eig_u[0])[la1:lfr],color = 'g',alpha=0.5)
plt.bar(np.arange(lfr,lres),np.real(eig_u[0])[lfr:lres],color = 'y',alpha=0.5)
plt.grid()
plt.title('Eigenmodes for $\lambda_1$',fontsize=20)
plt.xlabel('Clusters',fontsize=18)

plt.subplot(222)
plt.bar(np.arange(ld1),np.real(eig_u[1])[:ld1],color='b',alpha=0.5)
plt.bar(np.arange(ld1,ld2),np.real(eig_u[1])[ld1:ld2],color='c',alpha=0.5)
plt.bar(np.arange(ld2,la2),np.real(eig_u[1])[ld2:la2],color='m',alpha=0.5)
plt.bar(np.arange(la2,la1),np.real(eig_u[1])[la2:la1],color='r',alpha=0.5)
plt.bar(np.arange(la1,lfr),np.real(eig_u[1])[la1:lfr],color = 'g',alpha=0.5)
plt.bar(np.arange(lfr,lres),np.real(eig_u[1])[lfr:lres],color = 'y',alpha=0.5)
plt.grid()
plt.title('Eigenmodes for $\lambda_2$',fontsize=20)
plt.xlabel('Clusters',fontsize=18)


plt.subplot(223)
plt.bar(np.arange(ld1),np.real(eig_u[2])[:ld1],color='b',alpha=0.5)
plt.bar(np.arange(ld1,ld2),np.real(eig_u[2])[ld1:ld2],color='c',alpha=0.5)
plt.bar(np.arange(ld2,la2),np.real(eig_u[2])[ld2:la2],color='m',alpha=0.5)
plt.bar(np.arange(la2,la1),np.real(eig_u[2])[la2:la1],color='r',alpha=0.5)
plt.bar(np.arange(la1,lfr),np.real(eig_u[2])[la1:lfr],color = 'g',alpha=0.5)
plt.bar(np.arange(lfr,lres),np.real(eig_u[2])[lfr:lres],color = 'y',alpha=0.5)
plt.grid()
plt.title('Eigenmodes for $\lambda_3$',fontsize=20)
plt.xlabel('Clusters',fontsize=18)

plt.subplot(224)
plt.bar(np.arange(ld1),np.real(eig_u[3])[:ld1],color='b',alpha=0.5)
plt.bar(np.arange(ld1,ld2),np.real(eig_u[3])[ld1:ld2],color='c',alpha=0.5)
plt.bar(np.arange(ld2,la2),np.real(eig_u[3])[ld2:la2],color='m',alpha=0.5)
plt.bar(np.arange(la2,la1),np.real(eig_u[3])[la2:la1],color='r',alpha=0.5)
plt.bar(np.arange(la1,lfr),np.real(eig_u[3])[la1:lfr],color = 'g',alpha=0.5)
plt.bar(np.arange(lfr,lres),np.real(eig_u[3])[lfr:lres],color = 'y',alpha=0.5)
plt.grid()


plt.title('Eigenmodes for $\lambda_4$',fontsize=20)
plt.xlabel('Clusters',fontsize=18)

#### Eigenvector projections in the clusters

In [ ]:
new_order_array = np.concatenate(np.array([disp,circle,asym_fight,sym_fight,freeze, n_display]))

plt.figure(figsize=(16,6))

plt.subplot(131)
colors = cm.seismic((10*np.real(eig_u[1]))**3)
for i in range(len(res)):
    plt.scatter(qthr2[new_order_array[i]],qthr1[new_order_array[i]],color = colors[i])
plt.title('Eigenvalue 2',fontsize=16)


plt.subplot(132)
colors = cm.seismic(10*np.real(eig_u[2]))
for i in range(len(res)):
    plt.scatter(qthr2[new_order_array[i]],qthr1[new_order_array[i]],color = colors[i])
plt.title('Eigenvalue 3',fontsize=16)

plt.subplot(133)
colors = cm.seismic(10*np.real(eig_u[3]))
for i in range(len(res)):
    plt.scatter(qthr2[new_order_array[i]],qthr1[new_order_array[i]],color = colors[i])
plt.title('Eigenvalue 4',fontsize=16)


## Labelled Clusters

In [ ]:
plt.figure(figsize=(14,10))

plt.plot(pdisplay_c2,pdisplay_c1,'.',alpha=0.7,color='blue')
plt.plot(adisplay_c2,adisplay_c1,'.',alpha=0.7,color = 'cyan')
plt.plot(as_fight_c2,as_fight_c1,'.',alpha=0.7,color='magenta')
plt.plot(sym_fight_c2,sym_fight_c1,'.',alpha=0.7,color='red')
plt.plot(freeze_c2,freeze_c1,'.',alpha=0.7,color='green')
#plt.scatter(qthr1[c1[15]],-qthr2[c1[15]],alpha=0.7,color='k')
#plt.scatter(qthr1[c1[14]],-qthr2[c1[14]],alpha=0.7,color='k')
plt.plot(other_c2,other_c1,'.',alpha=0.7,color='goldenrod')

plt.xlabel('Z1',fontsize=20)
plt.ylabel('Z2',fontsize=20)

plt.legend(['Passive Display','Active Display','Aggressive state (asymmetric)','Aggressive state(symmetric)','Freeze','Other'],fontsize=16)